In [1]:
%load_ext autoreload
%autoreload 2
import torch, rdkit
import sys, pathlib
from pathlib import Path
PROJECT_ROOT = Path.home()/"바탕화면"/"torch"/"Chem"
sys.path.insert(0, str(PROJECT_ROOT))
from rdkit import Chem
from utils.utils import *


from tqdm import trange
from pathlib import Path
device   = "cuda"

vocab = dataset.vocab
index_to_token = {idx: token for token, idx in vocab.items()}

cuda


{'[#C]': 0, '[#N]': 1, '[*H0]': 2, '[=*H0]': 3, '[=Branch]': 4, '[=C]': 5, '[=N]': 6, '[=O]': 7, '[=S]': 8, '[Branch]': 9, '[C]': 10, '[Cl]': 11, '[EOS]': 12, '[F]': 13, '[NH0+1]': 14, '[N]': 15, '[OH0-1]': 16, '[O]': 17, '[PAD]': 18, '[PH1]': 19, '[P]': 20, '[Ring1]': 21, '[Ring2]': 22, '[SOS]': 23, '[S]': 24, '[SiH0]': 25, '[pop]': 26}
27
tensor([ 2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([23,  2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([ 2, 10, 17, 10,  2, 12, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])


In [2]:
def select_model(choice, latent):
    if choice == "Trans_MHA":
        from models.Trans_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda()

    # Trans
    elif choice == "Trans":
        from models.Trans import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda()

    # LSTM
    elif choice == "LSTM":
        from models.LSTM import CVAE, PriorNet
        model    = CVAE().cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda()

    # LSTM + MHA
    elif choice == "LSTM_MHA":
        from models.LSTM_MHA import CVAE, PriorNet
        model    = CVAE().cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda()

    return model, prior

def get_loss_fn(choice, latent):
    if choice == "Trans_MHA":
        loss_fn = ConditionalVAELoss(
            vocab_size=dataset.vocab_size,
            max_beta=0.7,
            anneal_steps=300,
            free_bits=0.01,
            capacity_max=1.0,
            capacity_inc=6e-4,
            gamma=5.0,
            prop_w=2.2,
            nce=0.2,
            sig_pen_p=0.02,
            sig_pen_q=0.05,
            imb=0.1,
            latent_dim=latent
        ).cuda()

    # Trans
    elif choice == "Trans":
        loss_fn = ConditionalVAELoss(
            vocab_size=dataset.vocab_size,
            max_beta=0.1,
            anneal_steps=300,
            cyc_steps=200,
            num_cycles=6,
            free_bits=0.06,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.8,
            prop_w=0.02,
            nce=0.1,
            sig_pen_p=0.003,
            sig_pen_q=0.001,
            imb=0.05,
            latent_dim=latent
        ).cuda()

    # LSTM
    elif choice == "LSTM":
        loss_fn = ConditionalVAELoss_LSTM(
            vocab_size=dataset.vocab_size,
            max_beta=0.08,
            anneal_steps=2400,
            free_bits=0.03,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.0,
            prop_w=3.0,
            nce=0.2,
            sig_pen_p=0.05,
            sig_pen_q=0.1,
            imb=0.5
        ).cuda()

    # LSTM + MHA
    elif choice == "LSTM_MHA":
        loss_fn = ConditionalVAELoss(
            vocab_size=dataset.vocab_size,
            max_beta=1.0,
            anneal_steps=2400,
            free_bits=0.03,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.0,
            prop_w=3.0,
            nce=0.2,
            sig_pen_p=0.3,
            sig_pen_q=0.0,
            imb=0.5
        ).cuda()
        
    return loss_fn

In [3]:
mode = "Trans"
latent_dim = 128
seq_len = dataset.max_len+3
model, prior = select_model(mode, latent_dim)
loss_fn = get_loss_fn(mode, latent_dim)

for head in [model.to_prop, model.to_prop_z]:
    nn.init.xavier_uniform_(head.weight)
    nn.init.zeros_(head.bias)

prop_params = list(model.to_prop.parameters())+list(model.to_prop_z.parameters())
prop_params_id = {id(p) for p in prop_params}
base_params = [p for p in model.parameters() if id(p) not in prop_params_id]
lr = 3e-4
lr_prior = lr*0.1
from torch.optim import AdamW
optim = AdamW([{'params':base_params, 'lr':lr},{'params':prop_params, 'lr':lr*0.5}])
optim2 = AdamW(prior.parameters(), lr=lr_prior)

In [4]:
import datetime
from rdkit import Chem, RDLogger
RDLogger.DisableLog('rdApp.error')
status_out = widgets.Output()

display(status_out)
epoch = 1200
model.train()
prior.train()
progress = tqdm(range(epoch), desc="Training")
loss_arr=[]

for i in progress:
    log_var_extract=[]
    log_var_p_extract = []
    batchloss = 0.0
    embeddings = []
    mean_extract = []
    results = []
    kld_raw_batch = 0.0
    bce = 0.0
    kld = 0.0
    if mode=="Trans" or mode=="Trans_MHA":
      for (smiles_enc, smiles_dec_input, smiles_dec_output, properties) in train_dataloader:
        optim.zero_grad()
        optim2.zero_grad()

        smiles_enc = smiles_enc.to(device)
        smiles_dec_input = smiles_dec_input.to(device)
        smiles_dec_output = smiles_dec_output.to(device)
        properties = properties.to(device)
        with torch.autocast("cuda"):
          output, tgt, means, log_var, tgt_z = model.forward(smiles_enc, smiles_dec_input, properties)
      
          mu_p, logvar_p = prior.forward(properties.squeeze())
          with torch.no_grad():
            log_var_extract.append(torch.exp(0.5 * log_var.cpu()).to(device))
            log_var_p_extract.append(torch.exp(0.5 * logvar_p.cpu()).to(device))
            mean_extract.append(means.cpu())
          
          loss, BCE, KLD, kld_raw, kld_per_token, prop = loss_fn.forward(output.float(),
                                  smiles_dec_output,
                                  means, log_var, mu_p, logvar_p,
                                  tgt, properties.float().squeeze(), tgt_z,
                                  i)
        
        results.append(output)

        loss.backward()
        optim.step()
        optim2.step()

        batchloss += loss.item()
        kld_raw_batch += kld_raw.item()
        bce += BCE.item()
        kld += KLD.item()
    else:
      for (smiles_dec_input, smiles_dec_output, properties) in train_dataloader_LSTM:
        tf_ratio = max(0.1, 1.0 - i / 300)
        optim.zero_grad()
        optim2.zero_grad()

        smiles_dec_input = smiles_dec_input.to(device)
        smiles_dec_output = smiles_dec_output.to(device)
        properties = properties.to(device)

        output, tgt, means, log_var, tgt_z = model.forward(smiles_dec_input, smiles_dec_output, properties, tf_ratio)
        
        mu_p, logvar_p = prior.forward(properties.squeeze())
        with torch.no_grad():
          log_var_extract.append(torch.exp(0.5 * log_var.cpu()).to(device))
          log_var_p_extract.append(torch.exp(0.5 * logvar_p.cpu()).to(device))
          mean_extract.append(means.cpu())
      
        loss, BCE, KLD, prop, kld_raw = loss_fn.forward(output.float(),
                              smiles_dec_output,
                              means, log_var, mu_p, logvar_p,
                              tgt, properties.float().squeeze(), tgt_z,
                              i)
        loss.backward()
        optim.step()
        optim2.step()

        batchloss += loss.item()
        kld_raw_batch += kld_raw.item()
        bce += BCE.item()
        kld += KLD.item()
        
    #Loss 값 추가
    loss = batchloss / len(train_dataloader)
    loss_arr.append(loss)

    # #Validity
    # results = torch.cat(results, dim=0).cpu()
    # results = nn.functional.softmax(results, dim=-1) 
    # argmax_indices = torch.argmax(results, dim=-1)
    
    # valid_smiles = []
    # for row in argmax_indices:
    #    smiles = tok_ids_to_smiles(row.tolist())
    #    valid_smiles.append(smiles or "")

    # valid_count = sum(bool(s) for s in valid_smiles)
    # valid_frac  = valid_count / len(valid_smiles)

    # 진행 바의 속성으로부터 필요한 값들 추출 (예시)
    elapsed = int(progress.format_dict.get("elapsed", 0))
    formatted_elap = str(datetime.timedelta(seconds=elapsed))
    rate = progress.format_dict.get("rate", None)
    sec_per_iter = 1 / rate if rate and rate != 0 else 0
    total = int(sec_per_iter * progress.total)
    formatted_total = str(datetime.timedelta(seconds=total))
    
    # 고정된 상태 정보를 업데이트 (Output 위젯에 출력)
    with status_out:
        clear_output(wait=True)
        print(f"🔹 Elapsed: {formatted_elap} | sec/iter: {sec_per_iter:.3f}s")
        print("🔹 Total time: ", formatted_total)
        print(f"🔹 Step: {i+1}/{progress.total}")
        print("🔹 loss: {:0.6f}".format(loss))
        print("[Posterior] sigma mean : {:0.6f}, ".format(torch.cat(log_var_extract).mean().item()),
              "sigma std : {:0.6f}".format(torch.cat(log_var_extract).std().mean()))
        print("[Prior]     sigma mean : {:0.6f}, ".format(torch.cat(log_var_p_extract).mean().item()),
              "sigma std : {:0.6f}".format(torch.cat(log_var_p_extract).std().mean()))
        print("raw KLD : ", kld_raw_batch / len(train_dataloader))
        print("Posterior μ per-dim std:", torch.cat(mean_extract,dim=0).view(-1, latent_dim).std(dim=0, unbiased=False).detach().cpu().numpy().mean())
        print("Posterior μ overall std:", torch.cat(mean_extract,dim=0).view(-1, latent_dim).std(unbiased=False).item())
        print("BCE : {:0.6f},".format(bce / seq_len / len(train_dataloader)),
              "KLD : {:0.6f},".format(kld / seq_len / len(train_dataloader)),
              "prop : {:0.6f},".format(prop))
        # print(f"Validity: {valid_frac:.2%}  ({valid_count}/{len(valid_smiles)})")

Output()

Training:   0%|          | 0/1200 [00:00<?, ?it/s]

In [5]:
model.eval()
prior.eval()
results = []
origin = []

properties_results=[]
properties_origin=[]
print(len(val_dataset))
with torch.no_grad():
    try:
        for (smiles_enc, smiles_dec_input, smiles_dec_output, properties) in val_dataloader:
            B = smiles_enc.size(0)

            smiles_enc = smiles_enc.to(device)
            smiles_dec_input = smiles_dec_input.to(device)
            smiles_dec_output = smiles_dec_output.to(device)
            properties = properties.to(device)

            result, tgt, means, log_var, z = model(smiles_enc, smiles_dec_input, properties)
            # result = model.deocder(smiles_dec_input, means.view(B, -1, latent_dim))
            # result = model.predict(result)

            results.append(result)
            origin.append(smiles_dec_output)

            properties_results.append(tgt)
            properties_origin.append(properties)
    except:
        for (smiles_dec_input, smiles_dec_output, properties) in val_dataloader_LSTM:

            smiles_dec_input = smiles_dec_input.to(device)
            smiles_dec_output = smiles_dec_output.to(device)
            properties = properties.to(device)

            result, tgt, means, log_var, z = model(smiles_dec_input, smiles_dec_output, properties)

            results.append(result)
            origin.append(smiles_dec_output)

            properties_results.append(tgt)
            properties_origin.append(properties)

results = torch.cat(results, dim=0)
origin = torch.cat(origin, dim=0)
results = nn.functional.softmax(results, dim=-1) 
argmax_indices = torch.argmax(results, dim=-1)
output = torch.nn.functional.one_hot(argmax_indices, num_classes=results.size(-1))
print(argmax_indices)
print(results.shape)
print(origin.shape)

from sklearn.metrics import mean_absolute_error
properties_origin=torch.cat(properties_origin,dim=0)
properties_results=torch.cat(properties_results,dim=0)
MAE_2 = mean_absolute_error(properties_origin.squeeze().cpu(), properties_results.squeeze().cpu())
print("MAE(properties) : ", MAE_2)


results_smiles = []
origin_smiles = []

for row in argmax_indices:
    smiles = tok_ids_to_smiles(row.tolist())
    results_smiles.append(smiles or "")

for row in origin:
    smiles = tok_ids_to_smiles(row.tolist())
    origin_smiles.append(smiles or "")

1567
tensor([[ 2, 10,  9,  ..., 10, 10, 10],
        [ 2, 15, 10,  ..., 12, 12, 12],
        [ 2, 17, 10,  ..., 10, 10, 10],
        ...,
        [ 2, 15, 10,  ...,  7,  7,  7],
        [ 2, 10, 15,  ..., 12, 12, 12],
        [ 2, 10, 15,  ..., 12, 12, 12]], device='cuda:0')
torch.Size([1567, 40, 27])
torch.Size([1567, 40])
MAE(properties) :  0.166839370684136


In [6]:
origin_smiles = [smiles.removesuffix("EOS").strip() for smiles in origin_smiles]
results_smiles = [smiles.removesuffix("EOS").strip() for smiles in results_smiles]

for i in range(len(results_smiles)):
    if(origin_smiles[i] != results_smiles[i]):
        print(i, "번째 다름!")
    print("real smiles      : ", origin_smiles[i])
    print("predicted smiles : ", results_smiles[i])


MAE = mean_absolute_error(origin.cpu(), torch.argmax(results.cpu(), dim=-1))
print("MAE : ", MAE)



0 번째 다름!
real smiles      :  [*]CC(C)OC(=O)OCC(C)N([*])CCC
predicted smiles :  [*]C(C)OC(=O)OC(C)N([*])CCC
1 번째 다름!
real smiles      :  [*]NC(=O)OC(CC)C([*])C(=O)OCC
predicted smiles :  [*]NC(=O)OC(C)C([*])C(=O)OCC
2 번째 다름!
real smiles      :  [*]OC(=O)OCC(C)NC(=O)C(C)C([*])CC
predicted smiles :  [*]OC(=O)OC(C)NC(=O)C(C)C([*])CC
3 번째 다름!
real smiles      :  [*]CNC(=O)CCCCNC(=O)NC([*])CC
predicted smiles :  [*]CNC(=O)CCNNNC(=O)NC([*])CC
4 번째 다름!
real smiles      :  [*]COC(=O)NC([*])C(=O)NCCCC
predicted smiles :  [*]CC(=O)NC([*])C(=O)NCCCC
5 번째 다름!
real smiles      :  [*]CNC(=O)COC(=O)OCC([*])C(C)C
predicted smiles :  [*]CNC(=O)COC(=O)OC([*])C(C)C
6 번째 다름!
real smiles      :  [*]OC(=O)OCC(O)=C(O)C([*])C=O
predicted smiles :  [*]OC(=O)OC(O)=C(O)C([*])C=O
7 번째 다름!
real smiles      :  [*]COC(=O)OCCCCC(=O)NC([*])C=C
predicted smiles :  [*]COC(=O)OCCCC(O)NC([*])C=C
real smiles      :  [*]CN(C)CCCNC(=O)OC([*])CSC
predicted smiles :  [*]CN(C)CCCNC(=O)OC([*])CSC
real smiles      :  [*]CNC(=O)OC(

In [7]:
from rdkit import Chem, RDLogger
from rdkit.Chem import DataStructs, rdFingerprintGenerator
RDLogger.DisableLog('rdApp.error')

generator = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

def tanimoto_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    fp1 = generator.GetFingerprint(mol1)
    fp2 = generator.GetFingerprint(mol2)
    return DataStructs.TanimotoSimilarity(fp1, fp2)

def is_valid(smiles):
    return Chem.MolFromSmiles(smiles) is not None

TS = 0.0
canbe = 0
notbe = 0

for sm, orig in zip(results_smiles, origin_smiles):
    if(len(sm)==0):
        notbe += 1
        continue
    if is_valid(sm) and is_valid(orig):
        sim = tanimoto_similarity(sm, orig)
        TS += sim
        canbe += 1
    else:
        notbe += 1

if canbe > 0:
    print("Tanimoto Similarity : ", TS / canbe)
else:
    print("No valid molecules to compare.")

print("가능한 분자 개수 :", canbe)
print("불가능한 분자 개수 :", notbe)
print("Valid fraction      :", canbe / len(results_smiles))


Tanimoto Similarity :  0.6137872630944933
가능한 분자 개수 : 1567
불가능한 분자 개수 : 0
Valid fraction      : 1.0


In [8]:
def save_weights(choice):
    if choice == "Trans_MHA":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256.pth")
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior.pth")        
        torch.save(prior.state_dict(), save_path)

    # Trans
    elif choice == "Trans":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256_no_mha.pth")
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior_no_mha.pth")        
        torch.save(prior.state_dict(), save_path)
    # LSTM
    elif choice == "LSTM":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM.pth")        
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_prior.pth")       
        torch.save(prior.state_dict(), save_path)


    # LSTM + MHA
    elif choice == "LSTM_MHA":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA.pth")        
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA_prior.pth")        
        torch.save(prior.state_dict(), save_path)

def select_model(choice, latent):
    if choice == "Trans_MHA":
        from models.Trans_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)

    # Trans
    elif choice == "Trans":
        from models.Trans import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256_no_mha.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior_no_mha.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    # LSTM
    elif choice == "LSTM":
        from models.LSTM import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)


    # LSTM + MHA
    elif choice == "LSTM_MHA":
        from models.LSTM_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    

    return model, prior

In [9]:
save_weights(mode)

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from rdkit import Chem, DataStructs

#####################################################################
# Utility: SMILES ↔ Fingerprint & Tanimoto
#####################################################################

def _smiles_to_fp(sm: str):
    mol = Chem.MolFromSmiles(sm)
    return Chem.RDKFingerprint(mol) if mol is not None else None


def tanimoto(sm1: str, sm2: str) -> float:
    fp1, fp2 = _smiles_to_fp(sm1), _smiles_to_fp(sm2)
    if fp1 is None or fp2 is None:
        return 0.0
    return DataStructs.TanimotoSimilarity(fp1, fp2)

#####################################################################
# Beam‑search decoder (temperature + length‑norm)
#####################################################################

def _beam_search_decode(model, z, sos_id: int, eos_id: int, *,
                        beam_width: int = 20, max_len: int = 128,
                        len_penalty: float = 1.0, vocab_size: int = 9999,
                        temperature: float = 0.9, alpha: float = 0.7):
    """Return list[List[int]] without SOS. OOV ids filtered."""
    device = z.device
    batch  = z.size(0)
    out_tokens = []
    for b in tqdm(range(batch)):
        beams = [([sos_id], 0.0, 0.0)]  # (seq, raw_logp, norm_score)
        finished = []
        for _ in range(max_len):
            new_beams = []
            for seq, raw_lp, _ in beams:
                if seq[-1] == eos_id:
                    finished.append((seq, raw_lp))
                    continue
                hidden = model.decoder(
                    torch.tensor(seq, device=device).unsqueeze(0),
                    z[b:b+1]
                )
                logits = model.predict(hidden)[:, -1] / temperature  # sharpen
                logp   = F.log_softmax(logits, dim=-1).squeeze(0)
                topk_val, topk_idx = torch.topk(logp, k=min(beam_width*2, logp.size(0)))
                # filter OOV
                cand = [(tid, tv) for tid, tv in zip(topk_idx.tolist(), topk_val.tolist()) if tid < vocab_size]
                for tid, tv in cand[:beam_width]:
                    new_seq   = seq + [tid]
                    new_rawlp = raw_lp + tv
                    norm_lp   = new_rawlp / (len(new_seq) ** alpha)
                    new_beams.append((new_seq, new_rawlp, norm_lp))
            if not new_beams:
                break
            new_beams.sort(key=lambda x: x[2], reverse=True)
            beams = new_beams[:beam_width]
        if finished:
            for seq, raw_lp in finished:
                beams.append((seq, raw_lp, raw_lp / (len(seq) ** alpha)))
        best_seq = max(beams, key=lambda x: x[1] / (len(x[0]) ** len_penalty))[0]
        # strip SOS/EOS
        best_seq = best_seq[1: best_seq.index(eos_id)] if eos_id in best_seq else best_seq[1:]
        out_tokens.append(best_seq)
    return out_tokens

#####################################################################
# Reconstruction with z = μ  (greedy or beam)
#####################################################################

def reconstruct_zmu(model, dataloader, vocab: dict, *,
                     beam_width: int = 1, len_penalty: float = 1.0,
                     max_len: int | None = None,
                     temperature: float = 0.9,
                     alpha: float = 0.7):
    """Return (mean_tanimoto, num_pairs). beam_width=1 → greedy."""
    model.eval()
    model.cpu()
    tanis, pairs = [], 0
    device = next(model.parameters()).device

    SOS, EOS, PAD = vocab['[SOS]'], vocab['[EOS]'], vocab['[PAD]']
    if max_len is None:
        max_len = 128

    for enc_in, _, _, props in dataloader:
        enc_in, props = enc_in.to(device), props.to(device)
        with torch.no_grad():
            prop_e  = model.input_embedding(props)
            encoded = model.encoder(enc_in, prop_e)
            mu_q    = model.to_means(encoded)
            z       = mu_q  # z = μ

            if beam_width == 1:
                # Greedy ------------------------------------------------------
                dec_in = torch.full((enc_in.size(0), 1), SOS, device=device)
                done   = torch.zeros(enc_in.size(0), dtype=torch.bool, device=device)
                out_tok= [[] for _ in range(enc_in.size(0))]
                for _ in range(max_len):
                    hidden = model.decoder(dec_in, z)
                    logits = model.predict(hidden)[:, -1] / temperature
                    next_tok = logits.argmax(-1, keepdim=True)
                    dec_in = torch.cat([dec_in, next_tok], dim=1)
                    for i, tok in enumerate(next_tok.squeeze(1).tolist()):
                        if not done[i]:
                            if tok == EOS:
                                done[i] = True
                            else:
                                out_tok[i].append(tok)
                    if done.all():
                        break
            else:
                # Beam --------------------------------------------------------
                out_tok = _beam_search_decode(model, z, SOS, EOS,
                                              beam_width=beam_width,
                                              max_len=max_len,
                                              len_penalty=len_penalty,
                                              vocab_size=len(vocab),
                                              temperature=temperature,
                                              alpha=alpha)
        # SELFIES → SMILES & Tanimoto ----------------------------------------
        for r_tok, o_tok in zip(enc_in.cpu().tolist(), out_tok):
            ref_sm = tok_ids_to_smiles([t for t in r_tok if t not in (PAD,)])
            out_sm = tok_ids_to_smiles([t for t in o_tok if t not in (PAD,)])
            if ref_sm and out_sm:
                tanis.append(tanimoto(ref_sm, out_sm))
                pairs += 1
    return np.mean(tanis) if tanis else 0.0, pairs


# 1) 길이 페널티 키워서 긴 후보 선호
print(reconstruct_zmu(model, val_dataloader, dataset.vocab,
                      beam_width=100, len_penalty=1.0, temperature=0.80,
                      max_len=dataset.max_len+3))

# 150/8 -> 0.4704

  0%|          | 0/256 [00:00<?, ?it/s]